This notebook will give an example how to use Renormalizer to calculate the ground state energy of the open boundary spin 1/2 1-D Heisenberg model.

$H = J \sum_i [S_i^z S_{i+1}^z + \frac{1}{2}(S_i^+ S_{i+1}^- + S_i^- S_{i+1}^+)]$

exact result (Bethe Anstatz):

L	     E/J

16	-6.9117371455749

24	-10.4537857604096

32	-13.9973156182243

48	-21.0859563143863

64	-28.1754248597421


In [5]:
from renormalizer.mps import Mps, Mpo, solver
from renormalizer.model import MolList2, ModelTranslator
from renormalizer.utils import basis as ba
from renormalizer.utils import Op
import numpy as np

# define the # of spins
nspin = 32

# define the model
# sigma^+ = S^+
# sigma^- = S^-
# 1/2 sigma^x,y,z = S^x,y,z

model = dict()
for ispin in range(nspin-1):
    model[(f"e_{ispin}", f"e_{ispin+1}")] = [(Op("sigma_z",0),
            Op("sigma_z",0), 1.0/4), (Op("sigma_+",0), Op("sigma_-",0), 1.0/2),
            (Op("sigma_-",0), Op("sigma_+", 0), 1.0/2)]

# set the spin order and local basis
order = {}
basis = []
for ispin in range(nspin):
    order[f"e_{ispin}"] = ispin
    basis.append(ba.BasisHalfSpin(sigmaqn=[0,0]))

# construct MPO
mol_list2 = MolList2(order, basis, model, ModelTranslator.general_model)
mpo = Mpo(mol_list2)
print(f"mpo_bond_dims:{mpo.bond_dims}")

# set the sweep paramter
M=30
procedure = [[M, 0.2], [M, 0], [M, 0], [M,0], [M,0]]

# initialize a random MPS
qntot = 0
mps = Mps.random(mol_list2, qntot, M)

mps.optimize_config.procedure = procedure
mps.optimize_config.method = "2site"

# optimize MPS
energies = solver.optimize_mps_dmrg(mps.copy(), mpo)
print("gs energy:", min(energies))


2020-06-03 17:27:55,796[DEBUG] # of operator terms: 93
2020-06-03 17:27:55,797[DEBUG] symbolic mpo algorithm: Hopcroft-Karp
2020-06-03 17:27:55,799[DEBUG] Input operator terms: 93
2020-06-03 17:27:55,802[DEBUG] After combination of the same terms: 93
2020-06-03 17:27:55,805[INFO] isite: 0, bipartite graph size: (4, 93)
2020-06-03 17:27:55,809[INFO] isite: 1, bipartite graph size: (7, 91)
2020-06-03 17:27:55,814[INFO] isite: 2, bipartite graph size: (8, 88)
2020-06-03 17:27:55,820[INFO] isite: 3, bipartite graph size: (8, 85)
2020-06-03 17:27:55,829[INFO] isite: 4, bipartite graph size: (8, 82)
2020-06-03 17:27:55,834[INFO] isite: 5, bipartite graph size: (8, 79)
2020-06-03 17:27:55,839[INFO] isite: 6, bipartite graph size: (8, 76)
2020-06-03 17:27:55,844[INFO] isite: 7, bipartite graph size: (8, 73)
2020-06-03 17:27:55,848[INFO] isite: 8, bipartite graph size: (8, 70)
2020-06-03 17:27:55,854[INFO] isite: 9, bipartite graph size: (8, 67)
2020-06-03 17:27:55,858[INFO] isite: 10, bipartit

mpo_bond_dims:[1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1]


2020-06-03 17:27:56,200[DEBUG] energy: -2.8187772413822825
2020-06-03 17:27:56,205[DEBUG] optimize site: 26, 27
2020-06-03 17:27:56,209[DEBUG] Hmat dim: 1920
2020-06-03 17:27:56,264[DEBUG] energy: -3.2294310464997484
2020-06-03 17:27:56,267[DEBUG] optimize site: 25, 26
2020-06-03 17:27:56,275[DEBUG] Hmat dim: 3600
2020-06-03 17:27:56,334[DEBUG] energy: -3.6955019137232314
2020-06-03 17:27:56,351[DEBUG] optimize site: 24, 25
2020-06-03 17:27:56,356[DEBUG] Hmat dim: 3600
2020-06-03 17:27:56,414[DEBUG] energy: -4.197754626511877
2020-06-03 17:27:56,423[DEBUG] optimize site: 23, 24
2020-06-03 17:27:56,425[DEBUG] Hmat dim: 3600
2020-06-03 17:27:56,467[DEBUG] energy: -4.547825052461047
2020-06-03 17:27:56,477[DEBUG] optimize site: 22, 23
2020-06-03 17:27:56,487[DEBUG] Hmat dim: 3600
2020-06-03 17:27:56,571[DEBUG] energy: -5.044532586707579
2020-06-03 17:27:56,583[DEBUG] optimize site: 21, 22
2020-06-03 17:27:56,590[DEBUG] Hmat dim: 3600
2020-06-03 17:27:56,767[DEBUG] energy: -5.4680661963630

gs energy: -13.997315092880086
